# Covid-19 projections based on data

Here I just want to fit a function to COVID-19 data, for both the accumulated cases as well as accumulated deaths:

$$ f(t, r, t_{peak}, N_{max}) = \frac{N_{max}}{1 + e^{-r\left(t-t_{peak}\right)}} $$

where $r$ is a number related to the rate of infection (it is *not* $R_0$!) $t_{peak}$ is the peak date, the day where there will be most infections and then the number of daily infections will start to fall, and $N_{max}$ is the forecast for the maximum number of cases or deaths. Of course these number will vary from day to day, as it was brilliantly pointed out by my friend [Dr. Christian Schuster](https://www.linkedin.com/in/c-schuster/) in [his post on LinkedIn](https://www.linkedin.com/pulse/c19-quo-vadis-cs-schuster/?trackingId=b6vLfaKTSme260vu6ugtbQ%3D%3D), which in fact is the mind that pointed me out how to perform this kind of forecast on top of the pandemics data.

In [1]:
import sys
import requests
import ipywidgets as widgets
from bs4 import BeautifulSoup
from ipywidgets import interact
import matplotlib.pyplot as plt
import numpy as np
import re
from scipy.optimize import curve_fit
%matplotlib inline

In [2]:
# get list of countries from main page
url = 'https://www.worldometers.info/coronavirus/'

res = requests.get(url)
html_page = res.content
soup = BeautifulSoup(html_page, 'html.parser')
text = soup.find_all('a', {'class' : 'mt_a'}) #text=True)

# store countries' names and urls in dict
countries = {t['href'].split('/')[1] : url + t['href'] for t in text}

In [10]:
# function to fit
def f(x, r, x_0, n):
    return n / (1. + np.exp(-r * (x - x_0)))

# do the curve plotting and fitting for each country
@interact
def showme(country = countries, fit_cases=False, fit_deaths=False):
    res_country = requests.get(country)
    html_country = res_country.content
    soup_country = BeautifulSoup(html_country, 'html.parser')
    data_country = soup_country.find_all('script', {'type' : 'text/javascript'})

    data = {}
    for d in data_country:
        m = re.search('series:\s+\[{\n\s+name:\s+\'(.*)\',\n(.*\n){2}\s+data:\s+\[(([0-9]+,)+[0-9]+)\]\s+}\]', str(d))
        if m:
            data[m[1]] = np.array(m[3].split(','), dtype=int)

    c = 'Country: {:s}'.format(country.split('/')[-2])
    for k in data.keys():
        print(c + ', found key \'{:s}\''.format(k))

    n = len(data)
        
    fig, ax = plt.subplots(ncols=n, figsize=(n * 5, 4))
    
    for i, d in enumerate(data):
        ax[i].plot(data[d], lw=3.5, color='blue')
        ax[i].set_title(d, fontsize=20)
        ax[i].tick_params(axis='both', which='major', labelsize=18)
    
    if fit_cases:
        x = np.arange(len(data['Cases']))
        y = data['Cases']
        p, _ = curve_fit(f, x, y, p0=[1., len(x), np.max(y)])
        x = np.arange(2 * len(x))
        y = f(x, p[0], p[1], p[2])
        for a in ax:
            if a.get_title() == 'Cases':
                a.plot(x, y, lw=2., ls='dotted', color='red')
                break
        print('\nFit params *Cases*\n > Rate = {:6.3f}\n > Peak day = {:6.3f}\n > Max cases = {:6.3f}'.format(p[0], p[1], p[2]))
    
    if fit_deaths:
        x = np.arange(len(data['Deaths']))
        y = data['Deaths']
        p, _ = curve_fit(f, x, y, p0=[1., len(x), np.max(y)])
        x = np.arange(2 * len(x))
        y = f(x, p[0], p[1], p[2])
        for a in ax:
            if a.get_title() == 'Deaths':
                a.plot(x, y, lw=2., ls='dotted', color='red')
                break
        print('\nFit params *Deaths*\n > Rate = {:6.3f}\n > Peak day = {:6.3f}\n > Max deaths = {:6.3f}'.format(p[0], p[1], p[2]))
    
    plt.suptitle('{:s}'.format(c), fontsize=28, y=1.1)
    plt.tight_layout()

interactive(children=(Dropdown(description='country', options={'us': 'https://www.worldometers.info/coronaviru…